Fabian Gonzalez Vera | A01367585

# Módulo 5
##  __[Project Block 2: Advanced Statistics](https://rpubs.com/cdorante/tc3006c_p1)__


In [44]:
# industria asignada - Manufacturing industries

Previos block data

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats.mstats import winsorize

In [46]:
dataf = pd.read_csv('dataus2023.csv') #Load dataset data
firmf = pd.read_csv('firmsus2023.csv')
print(dataf.shape, firmf.shape)

(533088, 22) (5615, 7)


Merge industry with data

In [47]:
dataf['qdate'] = pd.PeriodIndex(dataf.q, freq='Q')

In [48]:
firmf.columns

Index(['empresa', 'Nombre', 'status', 'partind', 'naics1', 'naics2',
       'SectorEconomatica'],
      dtype='object')

In [49]:
firms = firmf[['empresa','Nombre','status','naics1']] 
firms.columns=['firm','empresa','status', 'industria']

dataf = pd.merge(dataf, firms, on='firm', how='left') # Merge datasets
dataf.head()

,firm,q,fiscalmonth,revenue,cogs,sgae,otherincome,extraordinaryitems,finexp,incometax,...,adjprice,originalprice,sharesoutstanding,year,fixedassets,yearf,qdate,empresa,status,industria
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,63.33,104.00,"452,000.00",2000,NaN,NaN,2000Q1,"Agilent Technologies, Inc",activo,Industrias manufactureras
1,A,2000q2,6.00,"4,731,000.00","2,421,000.00","1,925,000.00","73,000.00",0.00,0.00,"161,000.00",...,44.91,73.75,"452,271.97",2000,"1,453,000.00","2,000.00",2000Q2,"Agilent Technologies, Inc",activo,Industrias manufactureras
2,A,2000q3,9.00,"7,401,000.00","3,790,000.00","3,016,000.00","101,000.00",0.00,0.00,"244,000.00",...,29.80,48.94,"453,014.58",2000,"1,581,000.00","2,000.00",2000Q3,"Agilent Technologies, Inc",activo,Industrias manufactureras
3,A,2000q4,12.00,"10,773,000.00","5,522,000.00","4,198,000.00","111,000.00",0.00,0.00,"407,000.00",...,33.34,54.75,"456,366.38",2000,"1,741,000.00","2,000.00",2000Q4,"Agilent Technologies, Inc",activo,Industrias manufactureras
4,A,2001q1,3.00,"2,841,000.00","1,449,000.00","1,113,000.00","19,000.00","-25,000.00",0.00,"119,000.00",...,18.71,30.73,"456,769.74",2001,"1,821,000.00","2,001.00",2001Q1,"Agilent Technologies, Inc",activo,Industrias manufactureras


In [50]:
dataf.columns

Index(['firm', 'q', 'fiscalmonth', 'revenue', 'cogs', 'sgae', 'otherincome',
       'extraordinaryitems', 'finexp', 'incometax', 'totalassets',
       'currentassets', 'inventory', 'totalliabilities', 'currentliabilities',
       'longdebt', 'adjprice', 'originalprice', 'sharesoutstanding', 'year',
       'fixedassets', 'yearf', 'qdate', 'empresa', 'status', 'industria'],
      dtype='object')

In [51]:
dataf.set_index(['firm','qdate'],inplace=True)
dataf['ry'] = np.log(dataf['adjprice']) - np.log(dataf.groupby(['firm'])['adjprice'].shift(4))

In [52]:

dataf['rq'] = np.log(dataf['adjprice']) - np.log(dataf.groupby(['firm'])['adjprice'].shift(1))

In [53]:
# Calculate future quarterly return

dataf['f1rq'] = dataf.groupby(['firm'])['rq'].shift(-1)
dataf['f4rq'] = dataf.groupby(['firm'])['rq'].shift(-4)

dataf['f1ry'] = dataf.groupby(['firm'])['ry'].shift(-1)
dataf['f4ry'] = dataf.groupby(['firm'])['ry'].shift(-4)

In [54]:
dataf[['rq','f1rq','ry','f4rq','f1ry','f4ry']].head(10)

rq  f1rq    ry  f4rq  f1ry  f4ry
firm qdate                                     
A    2000Q1   NaN -0.34   NaN -0.58   NaN -1.22
     2000Q2 -0.34 -0.41   NaN  0.06   NaN -0.82
     2000Q3 -0.41  0.11   NaN -0.51   NaN -0.92
     2000Q4  0.11 -0.58   NaN  0.38 -1.22 -0.65
     2001Q1 -0.58  0.06 -1.22  0.20 -0.82  0.13
     2001Q2  0.06 -0.51 -0.82 -0.39 -0.92 -0.32
     2001Q3 -0.51  0.38 -0.92 -0.59 -0.65 -0.40
     2001Q4  0.38  0.20 -0.65  0.32  0.13 -0.46
     2002Q1  0.20 -0.39  0.13 -0.31 -0.32 -0.98
     2002Q2 -0.39 -0.59 -0.32  0.40 -0.40 -0.19

In [55]:
dataf['ebit'] = dataf['revenue'] - dataf['cogs'] - dataf['sgae']
dataf['opm'] = np.where(dataf['revenue']==0, np.NaN, dataf['ebit']/dataf['revenue'])
dataf['netincome'] = dataf['ebit'] + dataf['otherincome'] + dataf['extraordinaryitems'] - dataf['finexp'] - dataf['incometax']
dataf['pm'] = np.where(dataf['revenue']==0, np.NaN, dataf['netincome']/dataf['revenue'])
dataf['ato'] = np.where(dataf['totalassets']==0, np.NaN, dataf['revenue']/dataf['totalassets'])
dataf['acidratio'] = np.where(dataf['currentliabilities']==0, np.NaN, dataf['currentassets']/dataf['currentliabilities'])
dataf['finlev'] = np.where(dataf['totalassets']==0, np.NaN, dataf['longdebt']/dataf['totalassets'])
dataf['mvalue'] = dataf['originalprice'] * dataf['sharesoutstanding']
dataf['bookvalue'] = dataf['totalassets'] - dataf['totalliabilities']

In [57]:
dataf.head()

q  fiscalmonth       revenue         cogs         sgae  \
firm qdate                                                                 
A    2000Q1  2000q1          NaN           NaN          NaN          NaN   
     2000Q2  2000q2         6.00  4,731,000.00 2,421,000.00 1,925,000.00   
     2000Q3  2000q3         9.00  7,401,000.00 3,790,000.00 3,016,000.00   
     2000Q4  2000q4        12.00 10,773,000.00 5,522,000.00 4,198,000.00   
     2001Q1  2001q1         3.00  2,841,000.00 1,449,000.00 1,113,000.00   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2    73,000.00                0.00    0.00 161,000.00 7,321,000.00   
     2000Q3   101,000.00                0.00    0.00 244,000.00 7,827,000.00   
     2000Q4   111,000.00                0.00    0.00 407,000.00 8,425,000.00   
     2001Q1    19,000.00          -25,000.00    0.00 119,000.00 9,208,000.00   

             ...  f4ry         ebit  opm  netincome   pm  ato  acidratio  \
firm qdate   ...                                                           
A    2000Q1  ... -1.22          NaN  NaN        NaN  NaN  NaN        NaN   
     2000Q2  ... -0.82   385,000.00 0.08 297,000.00 0.06 0.65       2.31   
     2000Q3  ... -0.92   595,000.00 0.08 452,000.00 0.06 0.95       2.13   
     2000Q4  ... -0.65 1,053,000.00 0.10 757,000.00 0.07 1.28       2.05   
     2001Q1  ...  0.13   279,000.00 0.10 154,000.00 0.05 0.31       1.78   

             finlev        mvalue    bookvalue  
firm qdate                                      
A    2000Q1     NaN 47,008,000.00          NaN  
     2000Q2    0.00 33,355,057.57 4,642,000.00  
     2000Q3    0.00 22,169,400.96 4,902,000.00  
     2000Q4    0.00 24,986,059.36 5,265,000.00  
     2001Q1    0.00 14,036,533.81 5,541,000.00  

[5 rows x 39 columns]

***
## 1.1 Descriptive Statistics


***
## 1.2 Multiple Regression - advanced topics

### 1.2.1 Calculation of Variables

In [88]:
# industria asignada - Manufacturing industries
manufacturyInds = dataf.loc[(dataf['industria']=='Industrias manufactureras')]

manufacturyInds['fsize'] = manufacturyInds.groupby(manufacturyInds['q'])['mvalue'].transform(lambda x: pd.qcut(x,3,labels=['small','medium','big']))
manufacturyInds['gsize'] = manufacturyInds['fsize']
manufacturyInds = pd.get_dummies(manufacturyInds,columns=['fsize'],drop_first=True)#,dummy_na=True)
manufacturyInds['oepsp'] = (manufacturyInds['ebit']/manufacturyInds['sharesoutstanding'])/manufacturyInds['originalprice']
manufacturyInds['epsp'] = (manufacturyInds['netincome']/manufacturyInds['sharesoutstanding'])/manufacturyInds['originalprice']
manufacturyInds['bmr'] = manufacturyInds['bookvalue']/manufacturyInds['mvalue']
MIy = manufacturyInds.loc[(manufacturyInds['status']=='activo') & (manufacturyInds['fiscalmonth']==12)]
MIy

q  fiscalmonth       revenue         cogs         sgae  \
firm qdate                                                                 
A    2000Q1  2000q1          NaN           NaN          NaN          NaN   
     2000Q2  2000q2         6.00  4,731,000.00 2,421,000.00 1,925,000.00   
     2000Q3  2000q3         9.00  7,401,000.00 3,790,000.00 3,016,000.00   
     2000Q4  2000q4        12.00 10,773,000.00 5,522,000.00 4,198,000.00   
     2001Q1  2001q1         3.00  2,841,000.00 1,449,000.00 1,113,000.00   
...             ...          ...           ...          ...          ...   
ZYNE 2022Q3  2022q3         9.00          0.00         0.00    26,565.56   
     2022Q4  2022q4        12.00          0.00         0.00    35,251.54   
     2023Q1  2023q1         3.00          0.00         0.00    10,471.17   
     2023Q2  2023q2         6.00          0.00         0.00    21,607.38   
     2023Q3  2023q3          NaN           NaN          NaN          NaN   

             otherincome  extraordinaryitems  finexp  incometax  totalassets  \
firm qdate                                                                     
A    2000Q1          NaN                 NaN     NaN        NaN          NaN   
     2000Q2    73,000.00                0.00    0.00 161,000.00 7,321,000.00   
     2000Q3   101,000.00                0.00    0.00 244,000.00 7,827,000.00   
     2000Q4   111,000.00                0.00    0.00 407,000.00 8,425,000.00   
     2001Q1    19,000.00          -25,000.00    0.00 119,000.00 9,208,000.00   
...                  ...                 ...     ...        ...          ...   
ZYNE 2022Q3      -893.80                0.00 -439.59       0.00    62,366.74   
     2022Q4      -631.13                0.00 -846.86       0.00    55,520.89   
     2023Q1       -89.98               -0.00 -447.31       0.00    48,636.53   
     2023Q2      -164.56                0.00 -882.03       0.00    40,476.56   
     2023Q3          NaN                 NaN     NaN        NaN          NaN   

             ...  acidratio  finlev        mvalue    bookvalue  gsize  \
firm qdate   ...                                                        
A    2000Q1  ...        NaN     NaN 47,008,000.00          NaN    big   
     2000Q2  ...       2.31    0.00 33,355,057.57 4,642,000.00    big   
     2000Q3  ...       2.13    0.00 22,169,400.96 4,902,000.00    big   
     2000Q4  ...       2.05    0.00 24,986,059.36 5,265,000.00    big   
     2001Q1  ...       1.78    0.00 14,036,533.81 5,541,000.00    big   
...          ...        ...     ...           ...          ...    ...   
ZYNE 2022Q3  ...       6.45    0.00     33,491.25    52,756.75  small   
     2022Q4  ...       5.97    0.00     24,943.24    46,228.76  small   
     2023Q1  ...       4.95    0.00     22,941.57    38,966.55  small   
     2023Q2  ...       3.43    0.00     17,490.71    29,020.57  small   
     2023Q3  ...        NaN     NaN     71,200.05          NaN  small   

             fsize_medium  fsize_big  oepsp  epsp  bmr  
firm qdate                                              
A    2000Q1         False       True    NaN   NaN  NaN  
     2000Q2         False       True   0.01  0.01 0.14  
     2000Q3         False       True   0.03  0.02 0.22  
     2000Q4         False       True   0.04  0.03 0.21  
     2001Q1         False       True   0.02  0.01 0.39  
...                   ...        ...    ...   ...  ...  
ZYNE 2022Q3         False      False  -0.79 -0.81 1.58  
     2022Q4         False      False  -1.41 -1.40 1.85  
     2023Q1         False      False  -0.46 -0.44 1.70  
     2023Q2         False      False  -1.24 -1.19 1.66  
     2023Q3         False      False    NaN   NaN  NaN  

[216105 rows x 45 columns]

In [93]:
import statsmodels.api as sm
# Add a column of 1's to the X dataframe in order to include the beta0 coefficient (intercept) in the model:
regm = MIy.copy()
regm = regm.dropna()
x = regm[['opm','pm','ato','revenue','totalassets','mvalue']]
X = sm.add_constant(x)

y = regm[['f1ry']]

mkmodel = sm.OLS(y,X,missing='drop').fit()

print(mkmodel.summary())

                            OLS Regression Results                            
Dep. Variable:                   f1ry   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     6.673
Date:                Thu, 21 Sep 2023   Prob (F-statistic):           4.59e-07
Time:                        10:55:11   Log-Likelihood:                -11126.
No. Observations:               12576   AIC:                         2.227e+04
Df Residuals:                   12569   BIC:                         2.232e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0135      0.006      2.407      